Connected to tf2_12 (Python 3.10.11)

In [2]:
import os
os.path.abspath("../../results/MAFN_new/")

'/mnt/project_mnt/teo_fs/rtorre/cernbox/git/GitHub/NormalizingFlows/NF4HEP/NormalizingFlowsHD/CMoG/results/MAFN_new'

In [8]:
def remove_and_create_files(root_folder, target_filename, new_filename):
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename == target_filename:
                file_path = os.path.join(dirpath, filename)
                print(f"Removing {file_path}")
                os.remove(file_path)
                
                # Create a new empty file in the same directory
                new_file_path = os.path.join(dirpath, new_filename)
                print(f"Creating {new_file_path}")
                with open(new_file_path, 'w') as f:
                    pass

root_folder = "../../results/RealNVPN_new/"
target_filename = "results_original.json"
new_filename = "done_new.txt"
remove_and_create_files(root_folder, target_filename, new_filename)

Removing ../../results/RealNVPN_new/run_83/results_original.json
Creating ../../results/RealNVPN_new/run_83/done_new.txt
Removing ../../results/RealNVPN_new/run_64/results_original.json
Creating ../../results/RealNVPN_new/run_64/done_new.txt
Removing ../../results/RealNVPN_new/run_45/results_original.json
Creating ../../results/RealNVPN_new/run_45/done_new.txt
Removing ../../results/RealNVPN_new/run_89/results_original.json
Creating ../../results/RealNVPN_new/run_89/done_new.txt
Removing ../../results/RealNVPN_new/run_26/results_original.json
Creating ../../results/RealNVPN_new/run_26/done_new.txt
Removing ../../results/RealNVPN_new/run_92/results_original.json
Creating ../../results/RealNVPN_new/run_92/done_new.txt
Removing ../../results/RealNVPN_new/run_219/results_original.json
Creating ../../results/RealNVPN_new/run_219/done_new.txt
Removing ../../results/RealNVPN_new/run_208/results_original.json
Creating ../../results/RealNVPN_new/run_208/done_new.txt
Removing ../../results/RealN

In [1]:
##############################################################################################
######################################### Initialize #########################################
##############################################################################################

visible_devices = [0,1,2]
import datetime
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing os...")
import os
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing sys...")
import sys
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing and initializing argparse...")
if not any("ipykernel" in arg for arg in sys.argv):
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("-v", "--visible_devices", help="Set visible devices", nargs='*', type=int, default=visible_devices)
    args = parser.parse_args()
    visible_devices = args.visible_devices if args.visible_devices else visible_devices
print("Visible devices:", visible_devices)
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing timer from timeit...")
from timeit import default_timer as timer
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Setting env variables for tf import (only device", visible_devices, "will be available)...")
os.environ["CUDA_VISIBLE_DEVICES"] = ','.join([str(i) for i in visible_devices])
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
#os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
#os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2'
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing numpy...")
import numpy as np
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing pandas...")
import pandas as pd
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing shutil...")
import shutil
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing subprocess...")
import subprocess
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing tensorflow...")
import tensorflow as tf
print("Tensorflow version:", tf.__version__)
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing tensorflow_probability...")
import tensorflow_probability as tfp
tfd = tfp.distributions
print("Tensorflow probability version:", tfp.__version__)
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing textwrap...")
import textwrap
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing timeit...")
from timeit import default_timer as timer
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing traceback...")
import traceback
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing typing...")
from typing import List, Tuple, Dict, Union, Optional, Any
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Setting tf configs...")
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for gpu_device in gpu_devices:
    tf.config.experimental.set_memory_growth(gpu_device, True)

print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing custom module...")

sys.path.append('../../../code')
import Bijectors, Distributions, MixtureDistributions, Plotters, Trainer, Utils # type: ignore
import GenerativeModelsMetrics as GMetrics # type: ignore

def get_gpu_info() -> Optional[List[str]]:
    try:
        gpu_info: str = subprocess.check_output(["nvidia-smi", "--query-gpu=gpu_name", "--format=csv,noheader"]).decode('utf-8')
        return gpu_info.strip().split('\n')
    except Exception as e:
        print(e)
        return None
gpu_models: Optional[List[str]] = get_gpu_info()
if gpu_models:
    training_device: str = gpu_models[0]
    print("Successfully loaded GPU model: {}".format(training_device))
else:
    training_device = 'undetermined'
    print("Failed to load GPU model. Defaulting to 'undetermined'.")
    
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "All modues imported successfully.")

##############################################################################################
####################################### Helper functions #####################################
##############################################################################################

def MixtureGaussian(ncomp: int,
                    ndims: int,
                    seed: int = 0) -> tfp.distributions.Mixture:
    targ_dist: tfp.distributions.Mixture = MixtureDistributions.MixMultiNormal1(ncomp,ndims,seed=seed)
    return targ_dist

def get_io_kwargs(path_to_results: str) -> Dict[str,Any]:
    return {'results_path': path_to_results,
            'load_weights': True,
            'load_results': False}
    
def get_data_kwargs(seed: int = 0) -> Dict[str,Any]:
    return {'seed': seed}

def get_compiler_kwargs(lr: float,
                        ignore_nans: bool,
                        nan_threshold: float
                       ) -> Dict[str,Any]:
    compiler_kwargs = {'optimizer': {'class_name': 'Custom>Adam', # this gives the new Adam optimizer
                                     'config': {'learning_rate': lr,
                                                'beta_1': 0.9,
                                                'beta_2': 0.999,
                                                'epsilon': 1e-07,
                                                'amsgrad': True}},
                       'metrics': [{'class_name': 'MinusLogProbMetric',
                                    'config': {'ignore_nans': ignore_nans, 
                                               'debug_print_mode': False}}],
                       'loss': {'class_name': 'MinusLogProbLoss', 
                                'config': {'name': "MLP", 
                                           'ignore_nans': ignore_nans, 
                                           'nan_threshold': nan_threshold, 
                                           'debug_print_mode': False}}}
    return compiler_kwargs
    
def get_callbacks_kwargs(checkpoint_path: str,
                         es_min_delta: float,
                         es_patience: int,
                         lr_reduce_factor: float,
                         lr_min_delta: float,
                         lr_patience: int,
                         min_lr: float
                         ) -> List[Dict[str,Any]]:
    callbacks_kwargs = [{'class_name': 'PrintEpochInfo',
                         'config': {}},
                        #{'class_name': 'HandleNaNCallback',
                        # 'config': {'checkpoint_path': checkpoint_path,
                        #            'lr_reduction_factor': lr_reduce_factor_on_nan,
                        #            'random_seed_var': np.random.randint(1000000)}},
                        #{'class_name': 'TerminateOnNaNFractionCallback',
                        # 'config': {'threshold': 0.1,
                        #            'validation_data': X_data_val}},
                        {'class_name': 'ModelCheckpoint',
                         'config': {'filepath': checkpoint_path,
                                    'monitor': 'val_loss',
                                    'save_best_only': True,
                                    'save_weights_only': True,
                                    'verbose': 1,
                                    'mode': 'auto',
                                    'save_freq': 'epoch'}},
                        {'class_name': 'EarlyStopping',
                         'config': {'monitor': 'val_loss', 
                                    'min_delta': es_min_delta, 
                                    'patience': es_patience, 
                                    'verbose': 1,
                                    'mode': 'auto', 
                                    'baseline': None, 
                                    'restore_best_weights': True}},
                        {'class_name': 'ReduceLROnPlateau', 
                         'config': {'monitor': 'val_loss', 
                                    'factor': lr_reduce_factor, 
                                    'min_delta': lr_min_delta, 
                                    'patience': lr_patience, 
                                    'min_lr': min_lr}},
                        {'class_name': 'TerminateOnNaN', 'config': {}}
                        ]
    return callbacks_kwargs

def get_fit_kwargs(batch_size: int,
                   epochs_input: int,
                   validation_data: Tuple[Union[np.ndarray,tf.Tensor],Union[np.ndarray,tf.Tensor]],
                   shuffle: bool,
                   verbose: int
                  ) -> Dict[str,Any]:
    fit_kwargs = {'batch_size': batch_size, 
                  'epochs': epochs_input, 
                  'validation_data': validation_data,
                  'shuffle': shuffle, 
                  'verbose': verbose}
    return fit_kwargs

def train_function(seeds: List[int],
                   nsamples: List[int],
                   run_number: int,
                   base_dist: tfp.distributions.Distribution,
                   targ_dist: tfp.distributions.Distribution,
                   hyperparams_dict: Dict[str, Any],
                   n_runs: int,
                   ndims: int,
                   bijector_name: str,
                   nbijectors: int,
                   spline_knots: Union[int,str],
                   range_min: int,
                   hidden_layers: List[int],
                   batch_size: int,
                   epochs_input: int,
                   lr_orig: float,
                   es_min_delta: float,
                   es_patience: int,
                   lr_reduce_factor: float,
                   lr_min_delta: float,
                   lr_patience: int,
                   min_lr: float,
                   activation: str,
                   regulariser: Optional[str],
                   eps_regulariser: float,
                   training_device: str,
                   path_to_results: str,
                   checkpoint_path: str,
                   max_retry: int,
                   debug_print_mode: bool,
                   nan_threshold: float,
                  ) -> Tuple[Dict[str, Any], Trainer.Trainer, int, float]:
    succeeded = False
    retry: int = 0
    lr: float = lr_orig
    while not succeeded:
        seed_train: int
        seed_test: int
        seed_dist: int
        seed_metrics: int
        seed_train, seed_test, seed_dist, seed_metrics = seeds
        seed_test = seed_train + 1                     
        Utils.reset_random_seeds(seed = seed_train)
        nsamples_train: int
        nsamples_val: int
        nsamples_test: int
        nsamples_train, nsamples_val, nsamples_test = nsamples
        X_data_train: tf.Tensor
        X_data_val: tf.Tensor
        Y_data_train: tf.Tensor
        Y_data_val: tf.Tensor
        X_data_train, X_data_val, Y_data_train, Y_data_val = Utils.generate_train_data(run_number = run_number,
                                                                                       targ_dist = targ_dist,
                                                                                       nsamples_train = nsamples_train,
                                                                                       nsamples_val = nsamples_val,
                                                                                       seed_train = seed_train)
        bijector: tfp.bijectors.Bijector = Bijectors.ChooseBijector(bijector_name = bijector_name,
                                                                    ndims = ndims,
                                                                    spline_knots = spline_knots,
                                                                    nbijectors = nbijectors,
                                                                    range_min = range_min,
                                                                    hidden_layers = hidden_layers,
                                                                    activation = activation,
                                                                    regulariser = regulariser,
                                                                    eps_regulariser = eps_regulariser)
        Utils.save_bijector_info(path_to_results, bijector)
        print("Building Trainer NFObject.\n")
        NFObject: Trainer.Trainer = Trainer.Trainer(base_distribution = base_dist,
                                                    flow = bijector, 
                                                    x_data_train = X_data_train,
                                                    y_data_train = Y_data_train,
                                                    io_kwargs = get_io_kwargs(path_to_results = path_to_results),
                                                    data_kwargs = get_data_kwargs(seed = seed_train),
                                                    compiler_kwargs = get_compiler_kwargs(lr = lr,
                                                                                          ignore_nans = True,
                                                                                          nan_threshold = nan_threshold),
                                                    callbacks_kwargs = get_callbacks_kwargs(checkpoint_path = checkpoint_path,
                                                                                            es_min_delta = es_min_delta,
                                                                                            es_patience = es_patience,
                                                                                            lr_reduce_factor = lr_reduce_factor,
                                                                                            lr_min_delta = lr_min_delta,
                                                                                            lr_patience = lr_patience,
                                                                                            min_lr = min_lr),
                                                    fit_kwargs = get_fit_kwargs(batch_size = batch_size,
                                                                                epochs_input = epochs_input,
                                                                                validation_data = (X_data_val, Y_data_val),
                                                                                shuffle = True,
                                                                                verbose = 2),
                                                    debug_print_mode = debug_print_mode)
        trainable_params: int = NFObject.trainable_params
        non_trainable_params: int = NFObject.non_trainable_params
        hyperparams_dict = Utils.update_hyperparams_dict(hyperparams_dict = hyperparams_dict,
                                                         run_number = run_number,
                                                         n_runs = n_runs,
                                                         seeds = [seed_train, seed_test, seed_dist, seed_metrics],
                                                         nsamples = [nsamples_train, nsamples_val, nsamples_test],
                                                         ndims = ndims,
                                                         corr = None,
                                                         bijector_name = bijector_name,
                                                         nbijectors = nbijectors,
                                                         spline_knots = spline_knots,
                                                         range_min = range_min,
                                                         hllabel = '-'.join(str(e) for e in hidden_layers),
                                                         trainable_parameters = trainable_params,
                                                         non_trainable_parameters = non_trainable_params,
                                                         batch_size = batch_size,
                                                         epochs_input = epochs_input,
                                                         activation = activation,
                                                         regulariser = regulariser,
                                                         eps_regulariser = eps_regulariser,
                                                         training_device = training_device)
        Utils.save_hyperparams_dict(path_to_results, hyperparams_dict)
        print(f"Training model with initial learning rate {lr}...")
        print("Train first sample:", X_data_train[0]) # type: ignore
        NFObject.train()
        training_time: float = NFObject.training_time # type: ignore
        loss_history = list(NFObject.history['loss'])
        if np.isnan(loss_history).any():
            print("The loss history contains NaN values.")

        if np.isinf(loss_history).any():
            print("The loss history contains Inf values.")

        if len(loss_history) > 0:
            if np.isnan(loss_history).any() or np.isinf(loss_history).any():
                succeeded = False
                seed_train = np.random.randint(1000000)
                lr = lr * lr_reduce_factor_on_nan
                retry = retry + 1
                print(f"Training failed: trying again with seed {seed_train} and lr {lr}.")
            else:
                succeeded = True
                print(f"Training succeeded with seed {seed_train}.")
        else:
            succeeded = False
            seed_train = np.random.randint(1000000)
            lr = lr * lr_reduce_factor_on_nan
            retry = retry + 1
            print(f"Training failed: trying again with seed {seed_train} and lr {lr}.")
            
        if retry > max_retry:
            raise Exception("Training failed for the maximum number of retry.")
        
    return hyperparams_dict, NFObject, seed_train, training_time # type: ignore
    

def prediction_function(hyperparams_dict: Dict[str, Any],
                        results_dict: Dict[str, Any],
                        gpu_models: Optional[List[str]],
                        NFObject: Trainer.Trainer,
                        ndims: int,
                        targ_dist: tfp.distributions.Distribution,
                        seed_test: int,
                        seed_metrics: int,
                        n_iter: int,
                        nsamples_test: int,
                        n_slices_factor: int,
                        dtype: type,
                        max_vectorize: int,
                        mirror_strategy: bool,
                        make_plots: bool,
                        path_to_results: str
                       ) -> Tuple[Dict[str, Any], GMetrics.TwoSampleTestInputs, int, float]:
    start_pred: float = timer()
    t_losses_all: list = list(NFObject.history['loss']) # type: ignore
    v_losses_all: list = list(NFObject.history['val_loss']) # type: ignore
    lr_all: list = list(NFObject.history['lr']) # type: ignore
    epochs_output: int = len(t_losses_all)
    training_time: float = NFObject.training_time # type: ignore
    #try:
    print("===========\nComputing predictions\n===========\n")
    print("Computing metrics...")
    start = timer()
    #raise Exception("Stopped before test.")
    DataInputs: GMetrics.TwoSampleTestInputs = GMetrics.TwoSampleTestInputs(dist_1_input = targ_dist,
                                                                            dist_2_input = NFObject.nf_dist,
                                                                            niter = n_iter,
                                                                            batch_size = nsamples_test,
                                                                            dtype_input = dtype,
                                                                            seed_input = seed_metrics,
                                                                            use_tf = True,
                                                                            mirror_strategy = mirror_strategy,
                                                                            verbose = True)
    #LRMetric: GMetrics.LRMetric = GMetrics.LRMetric(data_input = DataInputs,
    #                                                verbose = True)
    KSTest: GMetrics.KSTest = GMetrics.KSTest(data_input = DataInputs,
                                              verbose = True)
    SWDMetric: GMetrics.SWDMetric = GMetrics.SWDMetric(data_input = DataInputs,
                                                       verbose = True)
    FNMetric: GMetrics.FNMetric = GMetrics.FNMetric(data_input = DataInputs,
                                                    verbose = True)
    #LRMetric.compute()
    KSTest.compute(max_vectorize = max_vectorize)
    SWDMetric.compute(nslices = n_slices_factor*ndims)
    FNMetric.compute(max_vectorize = max_vectorize)
    #lr_result: Dict[str, np.ndarray] = LRMetric.Results[-1].result_value
    logprob_ref_ref_sum_list = None#lr_result["logprob_ref_ref_sum_list"].tolist()
    logprob_ref_alt_sum_list = None#lr_result["logprob_ref_alt_sum_list"].tolist()
    logprob_alt_alt_sum_list = None#lr_result["logprob_alt_alt_sum_list"].tolist()
    lik_ratio_list = None#lr_result["lik_ratio_list"].tolist()
    lik_ratio_norm_list = None#lr_result["lik_ratio_norm_list"].tolist()
    ks_result: Dict[str, np.ndarray] = KSTest.Results[-1].result_value
    ks_lists: List[List[float]] = ks_result["statistic_lists"].tolist()
    ks_means: List[float] = ks_result["statistic_means"].tolist()
    ks_stds: List[float] = ks_result["statistic_stds"].tolist()
    swd_result: Dict[str, np.ndarray] = SWDMetric.Results[-1].result_value
    swd_lists: List[List[float]] = swd_result["metric_lists"].tolist()
    swd_means: List[float] = swd_result["metric_means"].tolist()
    swd_stds: List[float] = swd_result["metric_stds"].tolist()
    fn_result: Dict[str, np.ndarray] = FNMetric.Results[-1].result_value
    fn_list: List[float] = fn_result["metric_list"].tolist()
    ad_lists: Optional[List[List[float]]] = None
    ad_means: Optional[List[float]] = None
    ad_stds: Optional[List[float]] = None
    wd_lists: Optional[List[List[float]]] = None
    wd_means: Optional[List[float]] = None
    wd_stds: Optional[List[float]] = None
    end = timer()
    metrics_time = end - start
    print(f"Metrics computed in {metrics_time:.2f} s.")
    #except:
    #    try:
    #        print("===========\nFailed on GPU, re-trying on CPU\n===========\n")
    #        with tf.device('/device:CPU:0'): # type: ignore
    #            print("Computing metrics...")
    #            start = timer()
    #            DataInputs = GMetrics.TwoSampleTestInputs(dist_1_input = targ_dist,
    #                                                      dist_2_input = NFObject.nf_dist,
    #                                                      niter = n_iter,
    #                                                      batch_size = nsamples_test,
    #                                                      dtype_input = dtype,
    #                                                      seed_input = seed_metrics,
    #                                                      use_tf = True,
    #                                                      verbose = True)
    #            LRMetric = GMetrics.LRMetric(data_input = DataInputs,
    #                                         verbose = True)
    #            KSTest = GMetrics.KSTest(data_input = DataInputs,
    #                                     verbose = True)
    #            SWDMetric = GMetrics.SWDMetric(data_input = DataInputs,
    #                                           verbose = True)
    #            FNMetric = GMetrics.FNMetric(data_input = DataInputs,
    #                                         verbose = True)
    #            LRMetric.compute()
    #            KSTest.compute(max_vectorize = max_vectorize)
    #            SWDMetric.compute(nslices = n_slices_factor*ndims)
    #            FNMetric.compute(max_vectorize = max_vectorize)
    #            lr_result = LRMetric.Results[-1].result_value
    #            logprob_ref_ref_sum_list = lr_result["logprob_ref_ref_sum_list"].tolist()
    #            logprob_ref_alt_sum_list = lr_result["logprob_ref_alt_sum_list"].tolist()
    #            logprob_alt_alt_sum_list = lr_result["logprob_alt_alt_sum_list"].tolist()
    #            lik_ratio_list = lr_result["lik_ratio_list"].tolist()
    #            lik_ratio_norm_list = lr_result["lik_ratio_norm_list"].tolist()
    #            ks_result = KSTest.Results[-1].result_value
    #            ks_lists = ks_result["statistic_lists"].tolist()
    #            ks_means = ks_result["statistic_means"].tolist()
    #            ks_stds = ks_result["statistic_stds"].tolist()
    #            swd_result = SWDMetric.Results[-1].result_value
    #            swd_lists = swd_result["metric_lists"].tolist()
    #            swd_means = swd_result["metric_means"].tolist()
    #            swd_stds = swd_result["metric_stds"].tolist()
    #            fn_result = FNMetric.Results[-1].result_value
    #            fn_list = fn_result["metric_list"].tolist()
    #            ad_lists = None
    #            ad_means = None
    #            ad_stds = None
    #            wd_lists = None
    #            wd_means = None
    #            wd_stds = None
    #            end = timer()
    #            metrics_time = end - start
    #            print(f"Metrics computed in {metrics_time:.2f} s.")
    #    except:
    #        print("===========\nFailed computing metrics\n===========\n")
    #        logprob_ref_ref_sum_list = []
    #        logprob_ref_alt_sum_list = []
    #        logprob_alt_alt_sum_list = []
    #        lik_ratio_list = []
    #        lik_ratio_norm_list = []
    #        ks_means = []
    #        ks_stds = []
    #        ks_lists = []
    #        ad_means = []
    #        ad_stds = []
    #        ad_lists = []
    #        fn_list = []
    #        wd_means = []
    #        wd_stds = []
    #        wd_lists = []
    #        swd_means = []
    #        swd_stds = []
    #        swd_lists = []
    #        metrics_time = 0.
    if make_plots:
        try:
            start = timer()
            Plotters.train_plotter(t_losses_all,v_losses_all,path_to_results)
            X_data_test: tf.Tensor = DataInputs.dist_1_num[:nsamples_test] # type: ignore
            X_data_nf: tf.Tensor = DataInputs.dist_2_num[:nsamples_test] # type: ignore
            Plotters.cornerplotter(X_data_test.numpy(),X_data_nf.numpy(),path_to_results,ndims,norm=True) # type: ignore
            Plotters.marginal_plot(X_data_test.numpy(),X_data_nf.numpy(),path_to_results,ndims) # type: ignore
            #Plotters.sample_plotter(X_data_test,nf_dist,path_to_results)
            end = timer()
            plots_time: float = end - start
            print(f"Plots done in {plots_time:.2f} s.")
        except Exception as ex:
            # Get current system exception
            ex_type, ex_value, ex_traceback = sys.exc_info()
            # Extract unformatter stack traces as tuples
            trace_back = traceback.extract_tb(ex_traceback) # type: ignore
            # Format stacktrace
            stack_trace = list()
            for trace in trace_back:
                stack_trace.append("File : %s , Line : %d, Func.Name : %s, Message : %s" % (trace[0], trace[1], trace[2], trace[3]))
            ex_type_str = f"Exception type: {ex_type.__name__}" # type: ignore
            print(textwrap.dedent(f"""\
                ===========
                print("===========\nFailed to plot\n===========\n")
                {ex_type_str}
                Exception message: {ex_value}
                Stack trace: {stack_trace}
                ===========
                """))
        
    end_pred: float = timer()
    prediction_time: float = end_pred - start_pred
    total_time: float = training_time + prediction_time
    results_dict = Utils.update_results_dict(results_dict = results_dict,
                                             hyperparams_dict = hyperparams_dict,
                                             train_loss_history = t_losses_all,
                                             val_loss_history = v_losses_all,
                                             lr_history = lr_all,
                                             epochs_output = epochs_output,
                                             training_time = training_time,
                                             prediction_time = prediction_time,
                                             total_time = total_time,
                                             logprob_ref_ref_sum_list = logprob_ref_ref_sum_list,
                                             logprob_ref_alt_sum_list = logprob_ref_alt_sum_list,
                                             logprob_alt_alt_sum_list = logprob_alt_alt_sum_list,
                                             lik_ratio_list = lik_ratio_list,
                                             lik_ratio_norm_list = lik_ratio_norm_list,
                                             ks_means = ks_means,
                                             ks_stds = ks_stds,
                                             ks_lists = ks_lists,
                                             ad_means = ad_means,
                                             ad_stds = ad_stds,
                                             ad_lists = ad_lists,
                                             fn_list = fn_list,
                                             wd_means = wd_means,
                                             wd_stds = wd_stds,
                                             wd_lists = wd_lists,
                                             swd_means = swd_means,
                                             swd_stds = swd_stds,
                                             swd_lists = swd_lists
                                             )
    return results_dict, DataInputs, prediction_time, total_time # type: ignore

##############################################################################################
################################## Parameters initialization #################################
##############################################################################################

### Initialize number of components ###
ncomp: int = 3

### Initialize hyperparameters lists ###
ndims_list: List[int] = [4, 8, 16, 32, 64, 100, 200, 400, 1000]
nbijectors_list: List[int] = [2]
hidden_layers_list: List[List[int]] = [[128, 128, 128], [256, 256, 256]]
seeds_list: List[int] = [0, 187, 377, 440, 520, 541, 721, 869, 926, 933]

### Initialize nsamples inputs ###
nsamples_train: int = 100000
nsamples_val: int = 30000
nsamples_test: int = 100000

### Initialize seeds inputs ###
seed_test: int = 0 # overwritten in the loop by seed_train + 1
seed_dist: int = 0
seed_metrics: int = seed_test

### Initialize bijector inputs ###
bijector_name: str = 'MsplineN'
range_min: int = -16
spline_knots_list: List[Union[int,str]] = [8, 12] # Only relevant for the neural spline

### Initialize NN hyperparameters ###
activation: str = 'relu'
regulariser: Optional[str] = None
eps_regulariser: float = 0.

### Initialzie training hyperparameters ###
epochs_input: int = 1
batch_size: int = 512
nan_threshold: float = 0.01
max_retry: int = 10
debug_print_mode: bool = True

### Initialize optimizer hyperparameters ###
lr_orig: float = 0.001

### Initialize callbacks hyperparameters ###
es_min_delta: float = .0001
es_patience: int = 100
lr_min_delta: float = .0001
lr_patience: int = 50
lr_reduce_factor: float = .5
lr_reduce_factor_on_nan: float = float(1/3)
min_lr: float = 1e-6

### Initialize parameters for inference ###
n_iter: int = 10
n_slices_factor: int = 2
dtype: type = tf.float32
max_vectorize: int = 10
mirror_strategy = True
make_plots = True

### Initialize old variables for backward compatibility
corr: Optional[str] = None

### Initialize dictionaries ###
results_dict: Dict[str, Any] = Utils.init_results_dict()
hyperparams_dict: Dict[str, Any] = Utils.init_hyperparams_dict()

### Initialize output dir ###
mother_output_dir: str = Utils.define_dir('../../results/MsplineN_new/')

### Create 'log' file ####
log_file_name: str = Utils.create_log_file(mother_output_dir, results_dict)

##############################################################################################
####################################### Training loop ########################################
##############################################################################################
run_to_evaluate = 321
run: int = 0
run_max: int = 1
run_number: int = 0
n_runs: int = len(ndims_list) * len(seeds_list) * len(nbijectors_list) * len(spline_knots_list) * len(hidden_layers_list)
start_global: float = timer()
for ndims in ndims_list:
    targ_dist: tfp.distributions.Distribution = MixtureGaussian(ncomp = ncomp, ndims = ndims, seed = seed_dist)
    base_dist: tfp.distributions.Distribution = Distributions.gaussians(ndims)
    for seed_train in seeds_list:
        for nbijectors in nbijectors_list:
            for spline_knots in spline_knots_list:
                for hidden_layers in hidden_layers_list:
                    if run > run_max:
                        raise Exception("Interrupted after one run.")
                    start_run: float = timer()
                    hllabel: str = '-'.join(str(e) for e in hidden_layers)
                    run_number = run_number + 1
                    results_dict_txt_saved: bool = False
                    results_dict_json_saved: bool = False
                    results_log_saved: bool = False
                    path_to_results: str
                    to_run: bool
                    path_to_results, to_run = Utils.define_run_dir(mother_output_dir+'run_'+str(run_number)+'/',
                                                                   force = "skip",
                                                                   bkp = False)
                    if run_number == run_to_evaluate:
                        to_run = True
                    if to_run:
                        if run_number == run_to_evaluate:
                            pass
                        else:
                            raise Exception(f"Run different than {run_to_evaluate}. Skipping.")
                        try:
                            dummy_file_path: str = os.path.join(path_to_results,'running.txt')
                            with open(dummy_file_path, 'w') as f:
                                pass
                            path_to_weights: str = Utils.define_dir(os.path.join(path_to_results, 'weights'))
                            checkpoint_path: str = os.path.join(path_to_weights, 'best_weights.h5')
                            ########### Model train ###########
                            NFObject: Trainer.Trainer
                            hyperparams_dict, NFObject, seed_train, training_time = train_function(seeds = [seed_train, seed_test, seed_dist, seed_metrics],
                                                                                                   nsamples = [nsamples_train, nsamples_val, nsamples_test],
                                                                                                   run_number = run_number,
                                                                                                   base_dist = base_dist,
                                                                                                   targ_dist = targ_dist,
                                                                                                   hyperparams_dict = hyperparams_dict,
                                                                                                   n_runs = n_runs,
                                                                                                   ndims = ndims,
                                                                                                   bijector_name = bijector_name,
                                                                                                   nbijectors = nbijectors,
                                                                                                   spline_knots = spline_knots,
                                                                                                   range_min = range_min,
                                                                                                   hidden_layers = hidden_layers,
                                                                                                   batch_size = batch_size,
                                                                                                   epochs_input = epochs_input,
                                                                                                   lr_orig = lr_orig,
                                                                                                   es_min_delta = es_min_delta,
                                                                                                   es_patience = es_patience,
                                                                                                   lr_reduce_factor = lr_reduce_factor,
                                                                                                   lr_min_delta = lr_min_delta,
                                                                                                   lr_patience = lr_patience,
                                                                                                   min_lr = min_lr,
                                                                                                   activation = activation,
                                                                                                   regulariser = regulariser,
                                                                                                   eps_regulariser = eps_regulariser,
                                                                                                   training_device = training_device,
                                                                                                   path_to_results = path_to_results,
                                                                                                   checkpoint_path = checkpoint_path,
                                                                                                   max_retry = max_retry,
                                                                                                   debug_print_mode = debug_print_mode,
                                                                                                   nan_threshold = nan_threshold)
                            
                            print(f"Model trained in {training_time:.2f} s.\n") # type: ignore
                            ########## Model prediction ###########
                            results_dict, DataInputs, prediction_time, total_time = prediction_function(hyperparams_dict = hyperparams_dict,
                                                                                                        results_dict = results_dict,
                                                                                                        gpu_models = gpu_models,
                                                                                                        NFObject = NFObject, # type: ignore
                                                                                                        ndims = ndims,
                                                                                                        targ_dist = targ_dist,
                                                                                                        seed_test = seed_test,
                                                                                                        seed_metrics = seed_metrics,
                                                                                                        n_iter = n_iter,
                                                                                                        nsamples_test = nsamples_test,
                                                                                                        n_slices_factor = n_slices_factor,
                                                                                                        dtype = dtype,
                                                                                                        max_vectorize = max_vectorize,
                                                                                                        mirror_strategy = mirror_strategy,
                                                                                                        make_plots = make_plots,
                                                                                                        path_to_results = path_to_results)
                            ########### Save results ###########
                            Utils.save_results_current_run_txt(path_to_results, results_dict)
                            results_dict_txt_saved = True
                            print("results.txt saved")
                            Utils.save_results_current_run_json(path_to_results, results_dict)
                            results_dict_json_saved = True
                            print("results.json saved")
                            Utils.save_results_log(log_file_name, results_dict)
                            results_log_saved = True
                            print("Results log saved")
                            print(f"Model predictions computed in {prediction_time:.2f} s.")
                            end_run: float = timer()
                            total_time_run=end_run-start_run
                            print(textwrap.dedent(f"""\
                                ===========
                                Run {run_number}/{n_runs} done in {total_time_run:.2f} s.
                                ===========
                                """))
                            run = run + 1
                            try:
                                os.remove(dummy_file_path)
                            except:
                                pass
                            dummy_file_path = os.path.join(path_to_results,'done.txt')
                            with open(dummy_file_path, 'w') as f:
                                pass
                        except Exception as ex:
                            try:
                                os.remove(dummy_file_path)
                            except:
                                pass
                            # Get current system exception
                            ex_type, ex_value, ex_traceback = sys.exc_info()
                            # Extract unformatter stack traces as tuples
                            trace_back = traceback.extract_tb(ex_traceback) # type: ignore
                            # Format stacktrace
                            stack_trace = list()
                            for trace in trace_back:
                                stack_trace.append("File : %s , Line : %d, Func.Name : %s, Message : %s" % (trace[0], trace[1], trace[2], trace[3]))
                            if not results_dict_txt_saved:
                                results_dict = Utils.update_results_dict(results_dict = results_dict,
                                                                         hyperparams_dict = hyperparams_dict)
                                Utils.save_results_current_run_txt(path_to_results, results_dict)
                            if not results_dict_json_saved:
                                Utils.save_results_current_run_json(path_to_results, results_dict)
                            if not results_log_saved:
                                Utils.save_results_log(log_file_name, results_dict)
                            ex_type_str = f"Exception type: {ex_type.__name__}" # type: ignore
                            print(textwrap.dedent(f"""\
                                ===========
                                Run {run_number}/{n_runs} failed.
                                {ex_type_str}
                                Exception message: {ex_value}
                                Stack trace: {stack_trace}
                                ===========
                                """))
                    else:
                        print(textwrap.dedent(f"""\
                            ===========
                            Run {run_number}/{n_runs} already exists. Skipping it.
                            ===========
                            """))
keys_to_remove = ['ks_lists', 'ad_lists', 'fn_list', 'wd_lists', 'swd_lists', 'train_loss_history', 'val_loss_history', 'lr_history']
dict_copy: Dict[str, Any] = {k: v for k, v in results_dict.items() if k not in keys_to_remove}
results_frame: pd.DataFrame = pd.DataFrame(dict_copy)
results_last_run_file: str = os.path.join(mother_output_dir,'results_last_run.txt')
results_frame.to_csv(results_last_run_file,index=False)
end_global: float = timer()
print(f"Everything done in {end_global-start_global:.2f} s.\n")

2023-10-11 17:31:55.050659: Importing os...
2023-10-11 17:31:55.050926: Importing sys...
2023-10-11 17:31:55.051002: Importing and initializing argparse...
Visible devices: [0, 1, 2]
2023-10-11 17:31:55.051235: Importing timer from timeit...
2023-10-11 17:31:55.051323: Setting env variables for tf import (only device [0, 1, 2] will be available)...
2023-10-11 17:31:55.051483: Importing numpy...
2023-10-11 17:31:55.193314: Importing pandas...
2023-10-11 17:31:55.382353: Importing shutil...
2023-10-11 17:31:55.382515: Importing subprocess...
2023-10-11 17:31:55.382580: Importing tensorflow...
Tensorflow version: 2.12.0
2023-10-11 17:31:57.722465: Importing tensorflow_probability...
Tensorflow probability version: 0.20.1
2023-10-11 17:31:58.095919: Importing textwrap...
2023-10-11 17:31:58.096011: Importing timeit...
2023-10-11 17:31:58.096079: Importing traceback...
2023-10-11 17:31:58.096141: Importing typing...
2023-10-11 17:31:58.096220: Setting tf configs...
2023-10-11 17:31:58.44789

Exception: Run different than 321. Skipping.

In [7]:
DataInputs: GMetrics.TwoSampleTestInputs = GMetrics.TwoSampleTestInputs(dist_1_input = targ_dist,
                                                                        dist_2_input = NFObject.nf_dist,
                                                                        niter = 10,
                                                                        batch_size = 100000,
                                                                        dtype_input = dtype,
                                                                        seed_input = seed_metrics,
                                                                        use_tf = True,
                                                                        mirror_strategy = True,
                                                                        verbose = True)

Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.


In [8]:
KSTest: GMetrics.KSTest = GMetrics.KSTest(data_input = DataInputs,
                                          verbose = True)

In [13]:
def Test_tf_new(self, max_vectorize: int = 100) -> None:
    """
    Function that computes the Kolmogorov-Smirnov test-statistic and p-value for two samples using tensorflow functions.
    The calculation is based in the custom function ks_2samp_tf.
    The calculation is performed in batches of size batch_size.
    The number of batches is niter.
    The total number of samples is niter*batch_size.
    The calculation is parallelized over max_vectorize (out of ndims*niter).
    The results are stored in the Results attribute.

    Parameters:
    ----------
    max_vectorize: int, optional, default = 100
        A maximum number of batch_size*max_vectorize samples per time are processed by the tensorflow backend.
        Given a value of max_vectorize, the ndims*niter KS calculations are split in chunks of max_vectorize.
        Each chunk is processed by the tensorflow backend in parallel. If ndims is larger than max_vectorize,
        the calculation is vectorized niter times over ndims.

    Returns:
    -------
    None
    """
    max_vectorize = int(max_vectorize)
    # Set alias for inputs
    if isinstance(self.Inputs.dist_1_num, np.ndarray):
        dist_1_num: tf.Tensor = tf.convert_to_tensor(self.Inputs.dist_1_num)
    else:
        dist_1_num = self.Inputs.dist_1_num # type: ignore
    if isinstance(self.Inputs.dist_2_num, np.ndarray):
        dist_2_num: tf.Tensor = tf.convert_to_tensor(self.Inputs.dist_2_num)
    else:
        dist_2_num = self.Inputs.dist_2_num # type: ignore
    if isinstance(self.Inputs.dist_1_symb, tfp.distributions.Distribution):
        dist_1_symb: tfp.distributions.Distribution = self.Inputs.dist_1_symb
    else:
        raise ValueError("dist_1_symb must be a tfp.distributions.Distribution object when use_tf is True.")
    if isinstance(self.Inputs.dist_2_symb, tfp.distributions.Distribution):
        dist_2_symb: tfp.distributions.Distribution = self.Inputs.dist_2_symb
    else:
        raise ValueError("dist_2_symb must be a tfp.distributions.Distribution object when use_tf is True.")
    ndims: int = self.Inputs.ndims
    niter: int
    batch_size: int
    niter, batch_size = [int(i) for i in self.get_niter_batch_size_tf()] # type: ignore
    dtype: tf.DType = tf.as_dtype(self.Inputs.dtype)
    seed: int = self.Inputs.seed
    
    # Utility functions
    def start_calculation() -> None:
        GMetrics.utils.conditional_tf_print(self.verbose, "\n------------------------------------------")
        GMetrics.utils.conditional_tf_print(self.verbose, "Starting KS tests calculation...")
        GMetrics.utils.conditional_tf_print(self.verbose, "Running TF KS tests...")
        GMetrics.utils.conditional_tf_print(self.verbose, "niter =", niter)
        GMetrics.utils.conditional_tf_print(self.verbose, "batch_size =", batch_size)
        self._start = timer()

    def end_calculation() -> None:
        self._end = timer()
        elapsed = self.end - self.start
        GMetrics.utils.conditional_tf_print(self.verbose, "KS tests calculation completed in", str(elapsed), "seconds.")
        
    def set_dist_num_from_symb(dist,
                               nsamples: int,
                               seed: int = 0
                              ) -> tf.Tensor:
        nonlocal dtype
        #dist_num: tf.Tensor = tf.cast(dist.sample(nsamples, seed = int(seed)), dtype = dtype) # type: ignore
        dist_num: tf.Tensor = GMetrics.utils.generate_and_clean_data(dist, nsamples, 1000, dtype = self.Inputs.dtype, seed = int(seed), mirror_strategy = True) # type: ignore
        return dist_num
    
    def return_dist_num(dist_num: tf.Tensor) -> tf.Tensor:
        return dist_num
    
    #@tf.function(experimental_compile=True)
    #@tf.function(reduce_retracing=True)
    def batched_test(start, end):
        GMetrics.utils.conditional_tf_print(tf.logical_and(tf.logical_or(tf.math.logical_not(tf.equal(start,0)),tf.math.logical_not(tf.equal(end,niter))), self.verbose), "Iterating from", start, "to", end, "out of", niter, ".")
        # Define unique constants for the two distributions. It is sufficient that these two are different to get different samples from the two distributions, if they are equal. 
        # There is not problem with subsequent calls to the batched_test function, since the random state is updated at each call.
        seed_dist_1  = int(1e6)  # Seed for distribution 1
        seed_dist_2  = int(1e12)  # Seed for distribution 2
        
        # Define batched distributions
        dist_1_k: tf.Tensor = tf.cond(tf.equal(tf.shape(dist_1_num[0])[0],0), # type: ignore
                                           true_fn = lambda: set_dist_num_from_symb(dist_1_symb, nsamples = batch_size*(end-start), seed = seed_dist_1),
                                           false_fn = lambda: return_dist_num(dist_1_num[start*batch_size:end*batch_size, :])) # type: ignore
        dist_2_k: tf.Tensor = tf.cond(tf.equal(tf.shape(dist_1_num[0])[0],0), # type: ignore
                                           true_fn = lambda: set_dist_num_from_symb(dist_2_symb, nsamples = batch_size*(end-start), seed = seed_dist_2),
                                           false_fn = lambda: return_dist_num(dist_2_num[start*batch_size:end*batch_size, :])) # type: ignore
        
        print("dist_1_k shape:", tf.shape(dist_1_k))
        print("Target shape:", (end-start, batch_size, ndims))
        
        dist_1_k = tf.reshape(dist_1_k, (end-start, batch_size, ndims))
        dist_2_k = tf.reshape(dist_2_k, (end-start, batch_size, ndims))
        # Define the loop body function
        def loop_body(args):
            idx1 = args[0]
            idx2 = args[1]
            metric, pvalue, _, _ = GMetrics.ks_metrics.ks_2samp_tf(dist_1_k[idx1, :, idx2], dist_2_k[idx1, :, idx2], verbose=False) # type: ignore
            metric = tf.cast(metric, dtype=dtype)
            pvalue = tf.cast(pvalue, dtype=dtype)
            return metric, pvalue
        
        # Create the range of indices for both loops
        indices = tf.stack(tf.meshgrid(tf.range(end-start), tf.range(ndims), indexing='ij'), axis=-1)
        indices = tf.reshape(indices, [-1, 2])
        
        # Use tf.vectorized_map to iterate over the indices
        statistic_lists, pvalue_lists = tf.vectorized_map(loop_body, indices) # type: ignore
        
        # Reshape the results back to (chunk_size, ndims)
        statistic_lists = tf.reshape(statistic_lists, (end-start, ndims))
        pvalue_lists = tf.reshape(pvalue_lists, (end-start, ndims))

        # Compute the mean values
        statistic_means = tf.cast(tf.reduce_mean(statistic_lists, axis=1), dtype=dtype)
        statistic_stds = tf.cast(tf.math.reduce_std(statistic_lists, axis=1), dtype=dtype)
        pvalue_means = tf.cast(tf.reduce_mean(pvalue_lists, axis=1), dtype=dtype)
        pvalue_stds = tf.cast(tf.math.reduce_std(pvalue_lists, axis=1), dtype=dtype)
        
        statistic_means = tf.expand_dims(statistic_means, axis=1)
        statistic_stds = tf.expand_dims(statistic_stds, axis=1)
        pvalue_means = tf.expand_dims(pvalue_means, axis=1)
        pvalue_stds = tf.expand_dims(pvalue_stds, axis=1)
        
        res = tf.concat([statistic_means, statistic_stds, statistic_lists, pvalue_means, pvalue_stds, pvalue_lists], axis=1)
    
        return res

    #@tf.function(experimental_compile=True)
    #@tf.function(reduce_retracing=True)
    def compute_test(max_vectorize: int = 100) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor, tf.Tensor, tf.Tensor, tf.Tensor]:
        # Check if numerical distributions are empty and print a warning if so
        GMetrics.utils.conditional_tf_print(tf.logical_and(tf.equal(tf.shape(dist_1_num[0])[0],0),self.verbose), "The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.") # type: ignore
        GMetrics.utils.conditional_tf_print(tf.logical_and(tf.equal(tf.shape(dist_1_num[0])[0],0),self.verbose), "The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.") # type: ignore
        
        # Ensure that max_vectorize is an integer larger than ndims
        max_vectorize = int(tf.cast(tf.maximum(max_vectorize, ndims),tf.int32)) # type: ignore

        # Compute the maximum number of iterations per chunk
        max_iter_per_chunk: int = int(tf.cast(tf.math.floor(max_vectorize / ndims), tf.int32)) # type: ignore

        # Compute the number of chunks
        nchunks: int = int(tf.cast(tf.math.ceil(niter / max_iter_per_chunk), tf.int32)) # type: ignore
        GMetrics.utils.conditional_tf_print(tf.logical_and(self.verbose,tf.logical_not(tf.equal(nchunks,1))), "nchunks =", nchunks)
        
        # Run the computation in chunks
        # Initialize the result TensorArray
        res = tf.TensorArray(dtype, size = nchunks)
        statistic_means = tf.TensorArray(dtype, size = nchunks)
        statistic_stds = tf.TensorArray(dtype, size = nchunks)
        statistic_lists = tf.TensorArray(dtype, size = nchunks)
        pvalue_means = tf.TensorArray(dtype, size = nchunks)
        pvalue_stds = tf.TensorArray(dtype, size = nchunks)
        pvalue_lists = tf.TensorArray(dtype, size = nchunks)

        def body(i, res):
            start = i * max_iter_per_chunk
            end = tf.minimum(start + max_iter_per_chunk, niter)
            chunk_result = batched_test(start, end) # type: ignore
            res = res.write(i, chunk_result)
            return i+1, res

        _, res = tf.while_loop(lambda i, res: i < nchunks, body, [0, res])
        
        for i in range(nchunks):
            res_i = res.read(i)
            statistic_means = statistic_means.write(i, res_i[:,0])
            statistic_stds = statistic_stds.write(i, res_i[:,1])
            statistic_lists = statistic_lists.write(i, res_i[:,2:2+ndims])
            pvalue_means = pvalue_means.write(i, res_i[:,2+ndims])
            pvalue_stds = pvalue_stds.write(i, res_i[:,3+ndims])
            pvalue_lists = pvalue_lists.write(i, res_i[:,4+ndims:])
            
        statistic_means_stacked = tf.reshape(statistic_means.stack(), (niter,))
        statistic_stds_stacked = tf.reshape(statistic_stds.stack(), (niter,))
        statistic_lists_stacked = tf.reshape(statistic_lists.stack(), (niter, ndims))
        pvalue_means_stacked = tf.reshape(pvalue_means.stack(), (niter,))
        pvalue_stds_stacked = tf.reshape(pvalue_stds.stack(), (niter,))
        pvalue_lists_stacked = tf.reshape(pvalue_lists.stack(), (niter, ndims))
        
        return statistic_means_stacked, statistic_stds_stacked, statistic_lists_stacked, pvalue_means_stacked, pvalue_stds_stacked, pvalue_lists_stacked
            
    start_calculation()
    
    Utils.reset_random_seeds(seed = seed)
    
    statistic_means, statistic_stds, statistic_lists, pvalue_means, pvalue_stds, pvalue_lists = compute_test(max_vectorize = max_vectorize) # type: ignore
                         
    end_calculation()
    
    timestamp: str = datetime.now().isoformat()
    test_name: str = "KS Test_np"
    parameters = {**self.param_dict, **{"backend": "tensorflow"}}
    result_value = {"statistic_lists": statistic_lists.numpy(),
                    "statistic_means": statistic_means.numpy(),
                    "statistic_stds": statistic_stds.numpy(),
                    "pvalue_lists": pvalue_lists.numpy(),
                    "pvalue_means": pvalue_means.numpy(),
                    "pvalue_stds": pvalue_stds.numpy()}
    result = TwoSampleTestResult(timestamp, test_name, parameters, result_value) # type: ignore
    self.Results.append(result)
KSTest.Test_tf = Test_tf_new.__get__(KSTest, GMetrics.KSTest)

In [14]:
KSTest.Test_tf(max_vectorize = 10)


------------------------------------------
Starting KS tests calculation...
Running TF KS tests...
niter = 10
batch_size = 100000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
nchunks = 10
Iterating from 0 to 1 out of 10 .
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')
dist_1_k shape: tf.Tensor([100000   1000], shape=(2,), dtype=int32)
Target shape: (<tf.Tensor: shape=(), dtype=int32, numpy=1>, 100000, 1000)
Iterating from 1 to 2 out of 10 .
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/repli

In [4]:
KSTest.compute(max_vectorize = 1)


------------------------------------------
Starting KS tests calculation...
Running TF KS tests...
niter = 10
batch_size = 10000
Tensor("mul:0", shape=(), dtype=int32)
Tensor("Shape_2:0", shape=(2,), dtype=int32)
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
Iterating from 0 to 1 out of 10 .
nchunks = 10


InvalidArgumentError: Graph execution error:

Detected at node 'Reshape' defined at (most recent call last):
    File "/local_data/scratch/rtorre/anaconda3/envs/tf2_12/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/local_data/scratch/rtorre/anaconda3/envs/tf2_12/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/local_data/scratch/rtorre/anaconda3/envs/tf2_12/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/local_data/scratch/rtorre/anaconda3/envs/tf2_12/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/local_data/scratch/rtorre/anaconda3/envs/tf2_12/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/local_data/scratch/rtorre/anaconda3/envs/tf2_12/lib/python3.10/site-packages/torna

In [14]:
prova = Utils.generate_and_clean_data(KSTest.Inputs.dist_2_input,
                                      1000,
                                      100, 
                                      dtype = KSTest.Inputs.dtype, 
                                      seed = int(KSTest.Inputs.seed), 
                                      mirror_strategy = False) # type: ignore

KeyboardInterrupt: 

In [14]:
len(prova)

100000

In [12]:
KSTest.Inputs.dist_2_input.sample(50)

<tf.Tensor: shape=(50, 1000), dtype=float32, numpy=
array([[8.051592 , 4.5737743, 4.9140997, ..., 3.57656  , 7.913433 ,
        6.203442 ],
       [6.7708473, 5.3443394, 5.3992977, ..., 4.2715263, 6.98471  ,
        6.9213696],
       [8.160212 , 4.7832146, 5.2080464, ..., 3.6006298, 8.060825 ,
        7.019944 ],
       ...,
       [8.28524  , 4.652333 , 5.267612 , ..., 3.6960063, 8.3304825,
        6.7796307],
       [5.8332796, 5.9626746, 5.3240795, ..., 8.798139 , 4.5733037,
        6.8851285],
       [5.393865 , 3.2084398, 5.251758 , ..., 4.878972 , 5.1827655,
        6.6090274]], dtype=float32)>

In [8]:
KSTest.Inputs.dist_2_input.sample(1)

<tf.Tensor: shape=(1, 1000), dtype=float32, numpy=
array([[ 6.45104170e+00,  1.68934059e+00,  5.01805258e+00,
         7.11854696e+00,  5.76389253e-01,  9.02331448e+00,
         5.01788568e+00,  5.07622242e-01,  2.07638168e+00,
         8.33012295e+00,  6.10408020e+00,  1.38044083e+00,
         3.54392028e+00,  5.45957899e+00, -1.15596056e-01,
         3.42731190e+00,  4.72251320e+00,  3.81470037e+00,
         7.34879112e+00,  7.59150076e+00,  2.12879753e+00,
         7.82541275e+00,  6.29458618e+00,  1.01701736e+01,
         7.47314453e+00,  8.11322021e+00,  5.59832859e+00,
         1.58839810e+00,  9.29706287e+00,  9.39323127e-01,
         5.16181564e+00,  1.39245522e+00,  8.82531452e+00,
         9.59433365e+00,  7.20421886e+00,  3.93652034e+00,
         3.03527951e+00,  3.04610515e+00,  1.38380134e+00,
         5.32061338e+00,  8.41667414e-01,  2.74513292e+00,
         6.29477310e+00,  7.28790236e+00,  2.07620144e-01,
         4.20255184e+00,  6.27298069e+00,  8.37334728e+00,
     